## 네이버 차량 제원 크롤링

In [1]:
import requests as req
from bs4 import BeautifulSoup
import pandas as pd
# 네이버 검색
url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query="

# 자동차 정보를 검색
def search_car(url, keyword):
    # 검색 URL 생성
    search_url = url + keyword + " 정보"
    # HTTP 요청 보내기
    response = req.get(search_url)
    # HTML 파싱
    soup = BeautifulSoup(response.text, "html.parser")
    # 정보 섹션 찾기
    info_section = soup.find(class_="info")
    # 만약 정보 섹션을 찾지 못했다면
    if info_section is None:
        print(f"{keyword}에 대한 정보를 찾을 수 없습니다.")
        return None, None, None, keyword
    # 정보 추출
    info = info_section.find_all("dt")
    info2 = info_section.find_all("dd")
    # span 태그 제거
    for infos in info2:
        span = infos.find("span")
        if span:
            span.decompose()
    # 검색 URL과 정보 반환
    return search_url, info, info2, keyword

# 데이터프레임을 CSV 파일로 저장
def save_to_csv(df, filename='car_info.csv'):
    df.to_csv(filename, mode='a', index=False, encoding="utf-8-sig")
# 빈 데이터프레임 생성
df_all = pd.DataFrame()

class_names={
    "0":"티볼리","1":"팰리세이드","2":"포터2","3":"i30","4":"G4렉스턴","5":"K3","6":"K5","7":"K7","8":"K9","9":"QM3","10":"QM6","11":"SM3","12":"SM5",
    "13":"SM6", "14":"SM7", "15":"XM3", "16":"그랜저", "17":"니로", "18":"다마스", "19":"레이", "20":"렉스턴스포츠","21":"맥스크루즈", "22":"모닝", 
    "23":"모하비", "24":"베뉴", "25":"벨로스터", "26":"봉고3", "27":"셀토스", "28":"스타렉스", "29":"스토닉", "30":"스팅어", "31":"스포티지",
    "32":"싼타페", "33":"쏘나타", "34":"쏘렌토", "35":"쏘울", "36":"아반떼", "37":"아이오닉", "38":"엑센트", "39":"카니발", "40":"코나",
    "41":"코란도", "42":"코란도투리스모", "43":"코란도C", "44":"투싼"
    }

In [2]:
# 클래스 이름 리스트 생성
class_name_list = list(class_names.values())
# 각 클래스 이름에 대해 자동차 정보 검색
for keyword in class_name_list:
    # 자동차 정보 검색 함수 호출
    urls, info, info2, _ = search_car(url, keyword)
    # 검색 결과가 없다면 다음 키워드로 넘어감
    if urls is None:
        continue
    # 정보를 딕셔너리로 페어링
    paired_data = {title: data for title, data in zip(info, info2)}
    # '차종' 정보 추가
    paired_data['차종'] = keyword
    # 딕셔너리를 DataFrame으로 변환
    df_temp = pd.DataFrame([paired_data])
    # DataFrame의 열 순서 재정렬
    cols = list(df_temp.columns)
    if '차종' in cols:
        cols.insert(0, cols.pop(cols.index('차종')))
        df_temp = df_temp.reindex(columns=cols)
    # 전체 DataFrame에 현재 DataFrame 추가
    df_all = pd.concat([df_all, df_temp], ignore_index=True)
# 결과를 CSV 파일로 저장
save_to_csv(df_all)
